In [0]:
%run ../Workspace/Shared/config/config

In [0]:
from datetime import date
from zipfile import ZipFile
import os

In [0]:
storage_landing = "landingbeca2026jan"
storage_datanexus = "storagedatanexus"

data_ingestao = date.today().strftime("%Y-%m-%d")

caminho_origem_balanca = f"abfss://balancacomercial@{storage_landing}.dfs.core.windows.net/"
caminho_origem_cnpj = f"abfss://cnpj@{storage_landing}.dfs.core.windows.net/"

caminho_raw = f"abfss://raw@storagedatanexus.dfs.core.windows.net/"

dbutils.fs.cp(caminho_origem_balanca, f"{caminho_raw}balancacomercial/data_ingestao={data_ingestao}", recurse=True)
dbutils.fs.cp(caminho_origem_cnpj, f"{caminho_raw}cnpj/data_ingestao={data_ingestao}", recurse=True)

In [0]:
# DESCOMPACTA ARQUIVOS DENTRO DA CAMADA RAW 

for file in dbutils.fs.ls(f'{caminho_raw}cnpj/data_ingestao={data_ingestao}'):

    caminho_zip_fonte = f"{caminho_raw}cnpj/data_ingestao={data_ingestao}/{file.name}"
    caminho_zip_temp = f"/dbfs/tmp/{file.name}"

    print(f'Controle, arquivo: {caminho_zip_fonte}')

    dbutils.fs.cp(caminho_zip_fonte, f"file:{caminho_zip_temp}", recurse=True)

    if file.name[-4:] == '.zip':
        with ZipFile(caminho_zip_temp, "r") as zip:
            nome_interno = zip.namelist()[0]
            zip.extract(nome_interno, "/dbfs/tmp")

        origem = f"file:/dbfs/tmp/{nome_interno}"

        dbutils.fs.cp(origem, caminho_raw+'cnpj_descompactados/'+file.name[:-4], recurse=True)
    
    os.remove(caminho_zip_temp)